In [1]:
import sqlite3
import sqlalchemy
import pandas as pd
import numpy as np
import warnings
import re
import sqlite3
warnings.filterwarnings("ignore")

In [2]:
#Load dataset from database
conn = sqlite3.connect("imdb_movies.sqlite")
df = pd.read_sql_query("SELECT * from marvel_movies_reddit", conn)
df.head()

,author,created_utc,domain,id,num_comments,score,selftext,title,url,datetime,movie
0,Angelus99King,1668125439,i.redd.it,yrw1ul,0,1,,Black Panther Wakanda Forever poster for the R...,https://i.redd.it/pejdnddur7z91.jpg,2022-11-11 00:10:39,black&panther&wakanda&forever
1,AGOTFAN,1668125768,wsj.com,yrw62i,2,1,,Black Panther Wakanda Forever Extends the Marv...,https://www.wsj.com/articles/black-panther-wak...,2022-11-11 00:16:08,black&panther&wakanda&forever
2,AGOTFAN,1668126038,wsj.com,yrw9ku,2,1,,Black Panther Wakanda Forever Extends the Marv...,https://www.wsj.com/articles/black-panther-wak...,2022-11-11 00:20:38,black&panther&wakanda&forever
3,EmceeMonte_CosplayDJ,1668126136,reddit.com,yrwaul,0,1,,self finished my Black Panther cosplay in time...,https://www.reddit.com/gallery/yrwaul,2022-11-11 00:22:16,black&panther&wakanda&forever
4,Present_Beautiful_25,1668126505,youtu.be,yrwffr,0,1,,Black Panther WAKANDA FOREVER Movie Review Spo...,https://youtu.be/DeETV1uHGKo,2022-11-11 00:28:25,black&panther&wakanda&forever


In [3]:
#Text preprocessing
def text_preprocessing(df):
    #Remove & character out of the name of movies
    df['movie'] = df['movie'].str.replace('&',' ',regex=False)
    #Combine title and selftext
    df['content'] = df['title'] + ' ' + df['selftext']
    #Converting text to lower case
    df['content'] = df['content'].apply(lambda x:x.lower())
    #Removing punctuation
    df['content'] = df['content'].str.replace('[^\w\s]',' ',regex=True)
    #Remove redundant spaces
    df['content'] = df['content'].str.replace('[\s]+',' ',regex=True)
    #Trim the space of texts
    df['content'] = df['content'].apply(lambda x:x.strip())

    return df

In [4]:
#Do sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def sentiment_analysis(df):
    #Text processing
    df = text_preprocessing(df)
    #Remove posts with content that have more than 3000 words
    df = df[df['content'].apply(lambda x:len(x.split())) <= 3000].reset_index(drop=True)
    #Do sentiment analysis for each content
    analyzer = SentimentIntensityAnalyzer()
    sentiments_df = pd.DataFrame(df['content'].apply(lambda x:analyzer.polarity_scores(x)).tolist())
    #Merge two dataframes
    final_df = pd.merge(df,sentiments_df,left_index=True,right_index=True)[['id','movie','content','datetime','compound','neg','neu','pos']]
    final_df['sentiment'] = 'neutral'
    final_df.loc[final_df['compound']>=0.05,'sentiment'] = 'positive'
    final_df.loc[final_df['compound']<=-0.05,'sentiment'] = 'negative'
    
    return final_df

In [5]:
sa_df = sentiment_analysis(df)
sa_df.head()

,id,movie,content,datetime,compound,neg,neu,pos,sentiment
0,yrw1ul,black panther wakanda forever,black panther wakanda forever poster for the r...,2022-11-11 00:10:39,0.0000,0.000,1.000,0.000,neutral
1,yrw62i,black panther wakanda forever,black panther wakanda forever extends the marv...,2022-11-11 00:16:08,0.6908,0.057,0.653,0.290,positive
2,yrw9ku,black panther wakanda forever,black panther wakanda forever extends the marv...,2022-11-11 00:20:38,0.2732,0.135,0.674,0.191,positive
3,yrwaul,black panther wakanda forever,self finished my black panther cosplay in time...,2022-11-11 00:22:16,0.0000,0.000,1.000,0.000,neutral
4,yrwffr,black panther wakanda forever,black panther wakanda forever movie review spo...,2022-11-11 00:28:25,0.5106,0.000,0.708,0.292,positive


In [11]:
#Load data to the database
def load(df):
    DATABASE_LOCATION = 'sqlite:///imdb_movies.sqlite'
    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect('imdb_movies.sqlite')
    cursor = conn.cursor()

    sql_query = """
    CREATE TABLE IF NOT EXISTS marvel_movies_reddit_sentiment(
        id VARCHAR,
        movie VARCHAR,
        content VARCHAR,
        datetime DATETIME,
        compound DOUBLE,
        neg DOUBLE,
        neu DOUBLE,
        pos DOUBLE,
        sentiment VARCHAR,
        CONSTRAINT primary_key_constraint PRIMARY KEY (id)
    )
    """
    cursor.execute(sql_query)
    df.to_sql('marvel_movies_reddit_sentiment', engine, index=False, if_exists='append')
    conn.close()

In [12]:
load(sa_df)